### Imports

In [1]:
import os
import sys
import yaml
import argparse
import numpy as np
import torch

from hls4ml.utils.config import config_from_pyg_model
from hls4ml.converters import convert_from_pyg_model
# module_path = os.path.abspath(os.path.join('../pyg_to_hls_hls4ml/hls4ml/utils'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from config import config_from_pyg_model

# module_path = os.path.abspath(os.path.join('../pyg_to_hls_hls4ml/hls4ml'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from converters import convert_from_pyg_model


from collections import OrderedDict
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# locals
from utils.models.interaction_network_pyg import InteractionNetwork
from model_wrappers import model_wrapper
from utils.data.dataset_pyg import GraphDataset
from utils.data.fix_graph_size import fix_graph_size

layer: Conv1d
func: <function parse_conv1d_layer at 0x7f1c8ba67f80>
layer: Conv2d
func: <function parse_conv2d_layer at 0x7f1c8ba70050>
layer: Linear
func: <function parse_linear_layer at 0x7f1c8ba70200>
layer: Softmax
func: <function parse_activation_layer at 0x7f1c8ba70170>
layer: ReLU
func: <function parse_activation_layer at 0x7f1c8ba70170>
layer: BatchNorm2d
func: <function parse_batchnorm_layer at 0x7f1c8ba700e0>
layer: BatchNorm1d
func: <function parse_batchnorm_layer at 0x7f1c8ba700e0>
layer: NodeBlock
func: <function parse_NodeBlock at 0x7f1c8ba7d290>
layer: EdgeBlock
func: <function parse_EdgeBlock at 0x7f1c8ba7d320>
layer: EdgeAggregate
func: <function parse_EdgeAggregate at 0x7f1c8ba7d3b0>
layer: ResidualBlock
func: <function parse_ResidualBlock at 0x7f1c8ba7d440>


### PyTorch Model

In [2]:
torch_model = InteractionNetwork(aggr="add", flow="source_to_target", hidden_size=40)
# torch_model_dict = torch.load("trained_models//IN_pyg_small_add_source_to_target_40_state_dict.pt")
# torch_model.load_state_dict(torch_model_dict)

for name, submodule in torch_model.named_modules():
    if name != "":
        print(f"{name}: {submodule}")

R1: RelationalModel(
  (layers): Sequential(
    (0): Linear(in_features=10, out_features=40, bias=True)
    (1): ReLU()
    (2): Linear(in_features=40, out_features=40, bias=True)
    (3): ReLU()
    (4): Linear(in_features=40, out_features=4, bias=True)
  )
)
R1.layers: Sequential(
  (0): Linear(in_features=10, out_features=40, bias=True)
  (1): ReLU()
  (2): Linear(in_features=40, out_features=40, bias=True)
  (3): ReLU()
  (4): Linear(in_features=40, out_features=4, bias=True)
)
R1.layers.0: Linear(in_features=10, out_features=40, bias=True)
R1.layers.1: ReLU()
R1.layers.2: Linear(in_features=40, out_features=40, bias=True)
R1.layers.3: ReLU()
R1.layers.4: Linear(in_features=40, out_features=4, bias=True)
O: ObjectModel(
  (layers): Sequential(
    (0): Linear(in_features=7, out_features=40, bias=True)
    (1): ReLU()
    (2): Linear(in_features=40, out_features=40, bias=True)
    (3): ReLU()
    (4): Linear(in_features=40, out_features=3, bias=True)
  )
)
O.layers: Sequential(
  (

### HLS Model

In [3]:
# forward_dict: defines the order in which graph-blocks are called in the model's 'forward()' method
forward_dict = OrderedDict()
forward_dict["R1"] = "EdgeBlock"
forward_dict["O"] = "NodeBlock"
forward_dict["res_block"] = "ResidualBlock"
forward_dict["R2"] = "EdgeBlock"

In [4]:
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
        "node_dim": 3,
        "edge_dim": 4
}

In [5]:
output_dir = "test_GNN"
config = config_from_pyg_model(torch_model,
                                   default_precision="ap_fixed<32,12>",
                                   default_index_precision='ap_uint<16>', 
                                   default_reuse_factor=1)
hls_model = convert_from_pyg_model(torch_model,
                                       n_edge=graph_dims['n_edge'],
                                       n_node=graph_dims['n_node'],
                                       edge_dim=graph_dims['edge_dim'],
                                       node_dim=graph_dims['node_dim'],
                                       forward_dictionary=forward_dict, 
                                       activate_final='sigmoid',
                                       output_dir=output_dir,
                                       hls_config=config)

rank: 2


In [6]:
output_dir = "test_GNN"
config = config_from_pyg_model(torch_model,
                                   default_precision="ap_fixed<32,12>",
                                   default_index_precision='ap_uint<16>', 
                                   default_reuse_factor=8)
print(f"config: {config}")
hls_model = convert_from_pyg_model(torch_model,
                                       n_edge=graph_dims['n_edge'],
                                       n_node=graph_dims['n_node'],
                                       edge_dim=graph_dims['edge_dim'],
                                       node_dim=graph_dims['node_dim'],
                                       forward_dictionary=forward_dict, 
                                       activate_final='sigmoid',
                                       output_dir=output_dir,
                                       hls_config=config)

config: {'Model': {'Precision': 'ap_fixed<32,12>', 'IndexPrecision': 'ap_uint<16>', 'ReuseFactor': 8, 'Strategy': 'Latency'}}
rank: 2


In [7]:
hls_model.compile()

Writing HLS project
layer name: node_attr. func: None
layer name: edge_attr. func: None
layer name: edge_index. func: None
layer name: R1. func: ['nnet::edgeblock<input2_t, input3_t, layer4_t, config4>(node_attr, edge_attr, edge_index, layer4_out, R1_w0, R1_b0, R1_w1, R1_b1, R1_w2, R1_b2, R1_w3, R1_b3);']
layer name: aggr5. func: ['nnet::edge_aggregate<input2_t, input3_t, layer5_t, aggregation_config5>(layer4_out, edge_index, layer5_out);']
nodeblock index: 6
nodeblock _function_template: nnet::nodeblock<{input_t}, {output_t}, {config}>({node_attr}, {edge_attr_aggr}, {out}, {w0}, {b0}, {w1}, {b1}, {w2}, {b2}, {w3}, {b3});
nodeblock final _function_template: nnet::nodeblock<input_t, layer6_t, config6>(node_attr, layer5_out, layer6_out, O_w0, O_b0, O_w1, O_b1, O_w2, O_b2, O_w3, O_b3);
layer name: O. func: ['nnet::nodeblock<input_t, layer6_t, config6>(node_attr, layer5_out, layer6_out, O_w0, O_b0, O_w1, O_b1, O_w2, O_b2, O_w3, O_b3);']
residualBlock template: nnet::{merge}<{input1_t}, {in

# Evaluation and prediction: hls_model.predict(input)

### Data

In [8]:
class data_wrapper(object):
    def __init__(self, node_attr, edge_attr, edge_index, target):
        self.x = node_attr
        self.edge_attr = edge_attr
        self.edge_index = edge_index.transpose(0,1)

        node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
        node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
        self.hls_data = [node_attr, edge_attr, edge_index]

        self.target = target
        self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs(graph_indir, graph_dims, n_graphs):
    graph_files = np.array(os.listdir(graph_indir))
    graph_files = np.array([os.path.join(graph_indir, graph_file)
                            for graph_file in graph_files])
    n_graphs_total = len(graph_files)
    IDs = np.arange(n_graphs_total)
    print(f"IDS: {IDs}")
    dataset = GraphDataset(graph_files=graph_files[IDs])

    graphs = []
    for data in dataset[:n_graphs]:
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, data.edge_attr, data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
#         if not bad_graph:
#             graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
        graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
    print(f"graphs length: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
        "node_dim": 3,
        "edge_dim": 4
    }
graphs = load_graphs(graph_indir, graph_dims, n_graphs=100)

IDS: [0 1 2 3 4 5 6 7 8 9]
graphs length: 10
writing test bench data for 1st graph


In [9]:
data = graphs[0]
print(f"graphs len: {len(graphs)}")
print(type(data.hls_data))
print(f"data.x shape:{data.x.shape}")
torch_pred = torch_model(data)
torch_pred = torch_pred.detach().cpu().numpy().flatten()
print(f"torch pred shape: {torch_pred.shape}")
hls_pred = hls_model.predict(data.hls_data)
print(f"hls_pred.shape: {hls_pred.shape}")
MSE = mean_squared_error(torch_pred, hls_pred)
print(f"MSE: {MSE}")

print((np.abs(torch_pred- hls_pred)))
print(torch_pred)
print(hls_pred)

graphs len: 10
<class 'list'>
data.x shape:torch.Size([28, 3])
torch pred shape: (37,)
hls_pred.shape: (37,)
MSE: 4.282571808289504e-06
[3.6702752e-03 3.6845803e-03 3.6625862e-03 3.6818981e-03 4.3183565e-05
 5.9455633e-05 4.4107437e-05 6.4074993e-05 3.8875043e-03 8.3446503e-07
 3.8879812e-03 2.5629997e-06 3.7461519e-05 3.5077333e-05 3.6597252e-05
 3.8325787e-05 3.8305521e-03 9.9629164e-05 9.7513199e-05 9.5009804e-05
 9.7095966e-05 3.8990676e-03 8.5830688e-05 8.6992979e-05 8.0704689e-05
 8.4966421e-05 3.8801134e-03 6.5803528e-05 6.9856644e-05 6.0498714e-05
 6.7293644e-05 3.8344860e-03 8.3237886e-05 8.5562468e-05 7.5519085e-05
 8.0585480e-05 3.8008392e-03]
[0.49242347 0.49240917 0.49243116 0.49241185 0.49214432 0.49212804
 0.4921434  0.49212343 0.49220625 0.49218833 0.49220577 0.49218494
 0.49215004 0.49215242 0.4921509  0.49214917 0.4922632  0.49208787
 0.49209    0.4920925  0.4920904  0.49219468 0.49210167 0.4921005
 0.4921068  0.49210253 0.49221364 0.4921217  0.49211764 0.492127
 0.49

In [10]:
def get_meanNstd(data, torch_pred, hls_pred):
#     print(data.x)
    node_mean = torch.mean(data.x)
    print(f"node_mean: {node_mean}")
    node_std = torch.std(data.x)
    print(f"node_std: {node_std}")
    node_abs_max = torch.max(torch.abs(data.x))
    print(f"node_abs_max: {node_abs_max}")
    edge_mean = torch.mean(data.edge_attr)
    print(f"edge_mean: {edge_mean}")
    edge_std = torch.std(data.edge_attr)
    print(f"edge_std: {edge_std}")
    edge_abs_max = torch.max(torch.abs(data.edge_attr))
    print(f"edge_abs_max: {edge_abs_max}")
    torch_pred_mean = np.mean(torch_pred)
    print(f"torch_pred_mean: {torch_pred_mean}")
    torch_pred_std = np.std(torch_pred)
    print(f"torch_pred_std: {torch_pred_std}")
    torch_pred_abs_max = np.max(np.abs(torch_pred))
    print(f"torch_pred_abs_max: {torch_pred_abs_max}")
    hls_pred_mean = np.mean(hls_pred)
    print(f"hls_pred_mean: {hls_pred_mean}")
    hls_pred_std = np.std(hls_pred)
    print(f"hls_pred_std: {hls_pred_std}")
    hls_pred_abs_max = np.max(np.abs(hls_pred))
    print(f"hls_pred_abs_max: {hls_pred_abs_max}")
    diff_mean = np.mean(torch_pred- hls_pred)
    print(f"diff_mean: {diff_mean}")
    diff_std = np.std(torch_pred- hls_pred)
    print(f"diff_std: {diff_std}")
    diff_abs_max = np.max(np.abs(torch_pred- hls_pred))
    print(f"diff_abs_max: {diff_abs_max}")
    return None

In [11]:
MSE_l = []
for data in graphs:
#     print(f"graphs len: {len(graphs)}")
#     print(type(data.hls_data))
#     print(f"data.x shape:{data.x.shape}")
    torch_pred = torch_model(data)
    torch_pred = torch_pred.detach().cpu().numpy().flatten()
#     print(f"torch pred shape: {torch_pred.shape}")
    hls_pred = hls_model.predict(data.hls_data)
#     print(f"hls_pred.shape: {hls_pred.shape}")
    MSE = mean_squared_error(torch_pred, hls_pred)
#     get_meanNstd(data, torch_pred, hls_pred)
#     print(f"MSE: {MSE}")
    MSE_l.append(MSE)
#     MAPE = mean_absolute_percentage_error(torch_pred, hls_pred)
#     print(f"MAPE: {MAPE}")

MSE_l = np.array(MSE_l)
print(f"overall MSE: {np.mean(MSE_l)}")
print(MSE_l.shape)

overall MSE: 8.005933523236308e-06
(10,)


In [12]:
for data_instance in data.hls_data:
    print(data_instance.shape)
    
print(data.edge_index.shape)

(28, 3)
(37, 4)
(37, 2)
torch.Size([2, 37])


In [13]:
# class data_wrapper_tau3mu:
#     def __init__(self, node_attr, edge_attr, edge_index, target):
#         self.x = node_attr
#         self.edge_attr = edge_attr
#         self.edge_index = edge_index.transpose(0,1)

#         node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
#         node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
#         self.hls_data = [node_attr, edge_attr, edge_index]

#         self.target = target
#         self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs_tau3mu(data_loader, graph_dims:dict):
    """
    params:
    dataloader: pyg dataloader with custom Tau3MuDataset as its dataset
    graph_dims: 
        graph_dims.keys = ["n_node": max number of nodes allowed in a graph/batch,
            "n_edge": max number of edges allowed in a graph/batch,
            "node_dim": feature dim of node,
            "edge_dim": feature dim of edge
        ]
    
    """

    graphs = []
    
    i = 0
    for data in data_loader:
        data.edge_index = data.edge_index.t()#transpose the edge_index
        n_edges = data.edge_attr.shape[0]
#         print((data.y.shape))
#         print(n_edges)
        data.y = data.y.expand(1, n_edges).flatten()
#         print(f"data.y.shape: {data.y.shape}")
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, 
                                                                             data.edge_attr, 
                                                                             data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
        target = torch.flatten(target)# flatten target to 1d array
        if not bad_graph:
            graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
        i +=1
        
    print(f"n_graphs: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"
# graph_indir = "/home/swissman777/projects/Tau3MuGNNs/data"
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
        "node_dim": 3,
        "edge_dim": 4
    }


In [14]:
import pickle as pkl
from torch_geometric.loader.dataloader import DataLoader
from utils.dataset import Tau3MuDataset

# with open('./trackml_data/data_loaders.pickle', 'rb') as file:
with open('./trackml_data/data_loaders_batch_size_1.pickle', 'rb') as file:
    data_loaders= pkl.load(file) #, x_dim, edge_attr_dim 

print(data_loaders)
# for stage in data_loaders.keys():
#     for data in data_loaders[stage]:
#         print(data.x.shape)
#         print(data.edge_index.shape)
#         print(data.edge_attr.shape)

# print(graphs)

{'train': <torch_geometric.loader.dataloader.DataLoader object at 0x7f1c5880e710>, 'valid': <torch_geometric.loader.dataloader.DataLoader object at 0x7f1c58804fd0>, 'test': <torch_geometric.loader.dataloader.DataLoader object at 0x7f1c58804790>}


In [15]:
from sklearn.preprocessing import StandardScaler
MSE_l = []
stages = ["train", "valid", "test"]
for stage in stages:
    graphs = load_graphs_tau3mu(data_loaders[stage], graph_dims)
    for data in graphs:
        torch_pred = torch_model(data)
        torch_pred = torch_pred.detach().cpu().numpy()
        hls_pred = hls_model.predict(data.hls_data)
        MSE = mean_squared_error(torch_pred, hls_pred)
    #     get_meanNstd(data,torch_pred, hls_pred)
    #     print(f"MSE: {MSE}")
        MSE_l.append(MSE)
    #     MAPE = mean_absolute_percentage_error(torch_pred, hls_pred)
    #     print(f"MAPE: {MAPE}")
MSE_l = np.array(MSE_l)
print(f"MSE means: {np.mean(MSE_l)}")

n_graphs: 332406
writing test bench data for 1st graph
n_graphs: 71226
writing test bench data for 1st graph
n_graphs: 175113
writing test bench data for 1st graph
MSE means: 3.849876520689577e-06


In [16]:
MSE_l.shape

(578745,)